In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy

# CHSH不等式

*使用時間の推定: Heron r2プロセッサで2分(注意: これは推定値です。実際の実行時間は異なる場合があります。)*

## 背景
このチュートリアルでは、Estimatorプリミティブを使用してCHSH不等式の破れを実証する実験を量子コンピューター上で実行します。

Clauser、Horne、Shimony、Holtにちなんで名付けられたCHSH不等式は、ベルの定理(1969年)を実験的に証明するために使用されます。この定理は、局所隠れた変数理論では量子力学におけるエンタングルメントのいくつかの結果を説明できないことを主張しています。CHSH不等式の破れは、量子力学が局所隠れた変数理論と矛盾することを示すために使用されます。これは量子力学の基礎を理解するための重要な実験です。

2022年のノーベル物理学賞は、Alain Aspect、John Clauser、Anton Zeilingerに授与されました。これは、彼らの量子情報科学における先駆的な業績、特にベルの不等式の破れを実証したエンタングル光子を用いた実験に対してです。

## 要件
このチュートリアルを始める前に、以下がインストールされていることを確認してください:

* Qiskit SDK v1.0以降、[visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)サポート付き
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22以降

## セットアップ

In [1]:
# General
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

## ステップ1: 古典的な入力を量子問題にマッピング
この実験では、エンタングルペアを作成し、各量子ビットを2つの異なる基底で測定します。最初の量子ビットの基底を$A$と$a$、2番目の量子ビットの基底を$B$と$b$とラベル付けします。これにより、CHSH量$S_1$を計算できます:

$$
S_1 = A(B-b) + a(B+b).
$$

各観測値は$+1$または$-1$です。明らかに、項$B\pm b$の1つは$0$で、もう1つは$\pm 2$です。したがって、$S_1 = \pm 2$です。$S_1$の平均値は次の不等式を満たす必要があります:

$$
|\langle S_1 \rangle|\leq 2.
$$

$S_1$を$A$、$a$、$B$、$b$の観点で展開すると:

$$
|\langle S_1 \rangle| = |\langle AB \rangle - \langle Ab \rangle + \langle aB \rangle + \langle ab \rangle| \leq 2
$$

別のCHSH量$S_2$を定義できます:

$$
S_2 = A(B+b) - a(B-b),
$$

これは別の不等式につながります:

$$
|\langle S_2 \rangle| = |\langle AB \rangle + \langle Ab \rangle - \langle aB \rangle + \langle ab \rangle| \leq 2
$$

量子力学が局所隠れた変数理論で記述できる場合、前述の不等式は真でなければなりません。しかし、このチュートリアルで実証されるように、これらの不等式は量子コンピューター上で破ることができます。したがって、量子力学は局所隠れた変数理論と互換性がありません。

理論についてもっと学びたい場合は、John Watrousによる[Entanglement in Action](/learning/courses/basics-of-quantum-information/entanglement-in-action/chsh-game)を探索してください。

量子コンピューター内の2つの量子ビット間にエンタングルペアを作成し、ベル状態$|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$を作成します。Estimatorプリミティブを使用すると、2つのCHSH量$\langle S_1\rangle$と$\langle S_2\rangle$の期待値を計算するために必要な期待値($\langle AB \rangle, \langle Ab \rangle, \langle aB \rangle$、および$\langle ab \rangle$)を直接取得できます。Estimatorプリミティブの導入前は、測定結果から期待値を構築する必要がありました。

2番目の量子ビットを$Z$基底と$X$基底で測定します。最初の量子ビットも直交基底で測定しますが、2番目の量子ビットに対して角度を持ち、$0$から$2\pi$の間でスイープします。ご覧のとおり、Estimatorプリミティブはパラメータ化された回路の実行を非常に簡単にします。一連のCHSH回路を作成する代わりに、測定角度を指定するパラメータと、パラメータの一連の位相値を持つ*1つ*のCHSH回路を作成するだけです。

最後に、結果を分析し、測定角度に対してプロットします。測定角度の特定の範囲で、CHSH量の期待値$|\langle S_1\rangle| > 2$または$|\langle S_2\rangle| > 2$となり、CHSH不等式の破れを実証できることがわかります。

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_kingston'

### Create a parameterized CHSH circuit

First, we write the circuit with the parameter $\theta$, which we call `theta`. The [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) can enormously simplify circuit building and output analysis by directly providing expectation values of observables. Many problems of interest, especially for near-term applications on noisy systems, can be formulated in terms of expectation values. `Estimator` (V2) primitive can automatically change measurement basis based on the supplied observable.

In [3]:
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif" alt="Output of the previous code cell" />

### パラメータ化されたCHSH回路を作成
まず、パラメータ$\theta$を持つ回路を記述します。これを`theta`と呼びます。[`Estimator`プリミティブ](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2)は、観測可能量の期待値を直接提供することで、回路構築と出力分析を大幅に簡素化できます。特にノイズのあるシステムでの短期的なアプリケーションにとって、多くの興味深い問題は期待値の観点から定式化できます。`Estimator` (V2)プリミティブは、提供された観測可能量に基づいて測定基底を自動的に変更できます。

In [4]:
number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
# Phases need to be expressed as list of lists in order to work
individual_phases = [[ph] for ph in phases]

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif)

### 後で割り当てる位相値のリストを作成
パラメータ化されたCHSH回路を作成した後、次のステップで回路に割り当てる位相値のリストを作成します。次のコードを使用して、$0$から$2 \pi$の範囲で等間隔の21個の位相値のリスト、つまり$0$、$0.1 \pi$、$0.2 \pi$、...、$1.9 \pi$、$2 \pi$を作成できます。

In [5]:
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)]
)

# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)]
)

### 観測可能量
期待値を計算する観測可能量が必要です。このケースでは、各量子ビットの直交基底を見ており、最初の量子ビットのパラメータ化された$Y-$回転により、2番目の量子ビット基底に対してほぼ連続的に測定基底をスイープできます。したがって、観測可能量$ZZ$、$ZX$、$XZ$、$XX$を選択します。

In [6]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

chsh_isa_circuit = pm.run(chsh_circuit)
chsh_isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif" alt="Output of the previous code cell" />

## ステップ2: 量子ハードウェア実行用に問題を最適化

総ジョブ実行時間を短縮するため、V2プリミティブはターゲットシステムがサポートする命令と接続性に準拠する回路と観測可能量(命令セットアーキテクチャ(ISA)回路と観測可能量と呼ばれる)のみを受け入れます。

### ISA回路

In [7]:
isa_observable1 = observable1.apply_layout(layout=chsh_isa_circuit.layout)
isa_observable2 = observable2.apply_layout(layout=chsh_isa_circuit.layout)

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif)

### ISA観測可能量

同様に、[`Runtime Estimator V2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2#run)でジョブを実行する前に、観測可能量をバックエンド互換にするために変換する必要があります。`SparsePauliOp`オブジェクトの`apply_layout`メソッドを使用して変換を実行できます。

In [8]:
# To run on a local simulator:
# Use the StatevectorEstimator from qiskit.primitives instead.

estimator = Estimator(mode=backend)

pub = (
    chsh_isa_circuit,  # ISA circuit
    [[isa_observable1], [isa_observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

job_result = estimator.run(pubs=[pub]).result()

## ステップ3: Qiskitプリミティブを使用して実行
[`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2)への1回の呼び出しで実験全体を実行するため、[Qiskit Runtime `Estimator`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2)プリミティブを作成して期待値を計算できます。`EstimatorV2.run()`メソッドは、`primitive unified blocs (PUBs)`のイテラブルを取ります。各PUBは`(circuit, observables, parameter_values: Optional, precision: Optional)`形式のイテラブルです。

In [9]:
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/f6267448-0.avif" alt="Output of the previous code cell" />

In the figure, the lines and gray areas delimit the bounds; the outer-most (dash-dotted) lines delimit the quantum-bounds ($\pm 2$), whereas the inner (dashed) lines delimit the classical bounds ($\pm 2\sqrt{2}$). You can see that there are regions where the CHSH witness quantities exceeds the classical bounds. Congratulations! You have successfully demonstrated the violation of CHSH inequality in a real quantum system!

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3xxAgm1SF1wGp9k)